### Backpropagation

We move backwards starting from the output layer and the loss function(SGD) cacluates the loss between actual values and preidcted and updates the weight accordingly at the output later    
ie if it is classification problem , then one of the node's value will increase which ever is the actual label and rest all values will be decreased.   
This process continues until we reach the input layer where we do not modify anything.

Steps are :
- Pass data to model via forward propogation (forward pass)
- Calculate loss on output
- SGD minimizes the loss
    - By calculating the gradient of the loss function and updating the weights
    - Gradient is calcualted via backpropogation

![](../resources/notations.png)

![](../resources/notation_2.png)